In [1]:
!pip install plotly-geo -q
!pip install geopandas -q

In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt

# Base path for where the USDA data has been stored
base_path = "/kaggle/input/cropnetv2/USDA"

def retrieve_county_data(years, crop_type, FIPS_code="01003"):
    all_data = []
    assert crop_type in ["Corn", "Cotton", "Soybeans", "WinterWheat"]
    
    if crop_type == "Soybeans":
        folder_name = "Soybeans"
        file_prefix = "Soybean"
    else:
        folder_name = crop_type
        file_prefix = crop_type
    
    path_template = "{}/{}/USDA_{}_County_{}.csv"
    for year in years:
        tmp_file_path = path_template.format(folder_name, year, file_prefix, year)
        tmp_path = os.path.join(base_path, tmp_file_path)
        try:
            data = pd.read_csv(tmp_path)
            all_data.append(data)
        except FileNotFoundError:
            print(f"File not found: {tmp_path}")
            continue
    df = pd.concat(all_data)
    county_data = df[(df['state_ansi'] == int(FIPS_code[:2])) & (df['county_ansi'] == int(FIPS_code[2:]))]
    return county_data

def get_production_yield_data(crop_type, years, FIPS_code):
    data = retrieve_county_data(years, crop_type, FIPS_code)
    if crop_type == 'Cotton':
        production_col = 'PRODUCTION, MEASURED IN 480 LB BALES'
        yield_col = 'YIELD, MEASURED IN LB / ACRE'
    else:
        production_col = 'PRODUCTION, MEASURED IN BU'
        yield_col = 'YIELD, MEASURED IN BU / ACRE'
    return data[['year', production_col, yield_col]]

# Set up the data
years = range(2016, 2023)
FIPS_code = "01003"  # Baldwin County, Alabama
crop_types = ["Corn", "Cotton", "Soybeans", "WinterWheat"]

# Retrieve data for each crop type
crop_data = {}
for crop in crop_types:
    crop_data[crop] = get_production_yield_data(crop, years, FIPS_code)

# Create the plots
fig, axs = plt.subplots(2, 4, figsize=(20, 10))
fig.suptitle("Crop Production and Yield in Baldwin County, AL (2016-2022)", fontsize=16)

for i, crop in enumerate(crop_types):
    data = crop_data[crop]
    
    # Production plot
    axs[0, i].plot(data['year'], data.iloc[:, 1], marker='o', color='blue')
    axs[0, i].set_title(f"{crop} Production")
    axs[0, i].set_xlabel("Year")
    if crop == 'Cotton':
        axs[0, i].set_ylabel("Production (480 lb bales)")
    else:
        axs[0, i].set_ylabel("Production (bushels)")
    axs[0, i].grid(True)
    
    # Yield plot
    axs[1, i].plot(data['year'], data.iloc[:, 2], marker='o', color='green')
    axs[1, i].set_title(f"{crop} Yield")
    axs[1, i].set_xlabel("Year")
    if crop == 'Cotton':
        axs[1, i].set_ylabel("Yield (lb/acre)")
    else:
        axs[1, i].set_ylabel("Yield (bu/acre)")
    axs[1, i].grid(True)

plt.tight_layout()
plt.subplots_adjust(top=0.92)  # Adjust to make room for suptitle
plt.show()

# Print the data
for crop in crop_types:
    print(f"\n{crop} Production and Yield Data:")
    print(crop_data[crop])

In [ ]:
import os
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from urllib.request import urlopen
import json

# Base path for where the USDA data has been stored
base_path = "/kaggle/input/cropnetv2/USDA"

def retrieve_state_data(year, crop_type):
    assert crop_type in ["Corn", "Cotton", "Soybeans", "WinterWheat"]
    
    if crop_type == "Soybeans":
        folder_name = "Soybeans"
        file_prefix = "Soybean"
    else:
        folder_name = crop_type
        file_prefix = crop_type
    
    file_path = f"{folder_name}/{year}/USDA_{file_prefix}_County_{year}.csv"
    full_path = os.path.join(base_path, file_path)
    
    try:
        data = pd.read_csv(full_path)
        alabama_data = data[data['state_name'] == 'ALABAMA']
        return alabama_data
    except FileNotFoundError:
        print(f"File not found: {full_path}")
        return pd.DataFrame()

def create_choropleth(year, crop_type, data_type='production'):
    # Retrieve data
    df = retrieve_state_data(year, crop_type)
    
    if df.empty:
        print(f"No data available for {crop_type} in {year}")
        return
    
    # Prepare data for choropleth
    if crop_type == 'Cotton':
        production_col = 'PRODUCTION, MEASURED IN 480 LB BALES'
        yield_col = 'YIELD, MEASURED IN LB / ACRE'
    else:
        production_col = 'PRODUCTION, MEASURED IN BU'
        yield_col = 'YIELD, MEASURED IN BU / ACRE'
    
    if data_type == 'production':
        values = df[production_col]
        colorbar_title = 'Production'
    else:
        values = df[yield_col]
        colorbar_title = 'Yield'
    
    fips = df['state_ansi'].astype(str).str.zfill(2) + df['county_ansi'].astype(str).str.zfill(3)
    
    # Load GeoJSON for Alabama counties
    with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
        counties = json.load(response)
    
    # Filter for only Alabama counties
    alabama_counties = [county for county in counties['features'] if county['properties']['STATE'] == '01']
    alabama_geojson = {'type': 'FeatureCollection', 'features': alabama_counties}
    
    # Create custom color scale
    min_value, max_value = values.min(), values.max()
    bins = np.linspace(min_value, max_value, 6)
    colorscale = [
        [0, "rgb(247,251,255)"],
        [0.2, "rgb(198,219,239)"],
        [0.4, "rgb(107,174,214)"],
        [0.6, "rgb(66,146,198)"],
        [0.8, "rgb(33,113,181)"],
        [1, "rgb(8,69,148)"]
    ]
    
    # Create choropleth map
    fig = go.Figure(go.Choroplethmapbox(
        geojson=alabama_geojson,
        locations=fips,
        z=values,
        colorscale=colorscale,
        zmin=min_value,
        zmax=max_value,
        marker_opacity=0.5,
        marker_line_width=0,
        colorbar=dict(
            title=colorbar_title,
            tickvals=bins,
            ticktext=[f"{x:.0f}" for x in bins]
        )
    ))
    
    fig.update_layout(
        mapbox_style="carto-positron",
        mapbox_zoom=6,
        mapbox_center={"lat": 32.7794, "lon": -86.8287},
        title=f'{crop_type} {colorbar_title} in Alabama ({year})',
        width=800,
        height=600
    )
    
    fig.show()

# Example usage
create_choropleth(2022, 'Cotton', 'production')
create_choropleth(2022, 'Corn', 'yield')

In [2]:
import os
import pandas as pd

def get_counties_with_full_data(crop_type, state_fips, start_year=2017, end_year=2022):
    base_path = "/kaggle/input/cropnetv2/USDA"
    
    assert crop_type in ["Corn", "Cotton", "Soybeans", "WinterWheat"]
    
    if crop_type == "Soybeans":
        folder_name = "Soybeans"
        file_prefix = "Soybean"
    else:
        folder_name = crop_type
        file_prefix = crop_type
    
    path_template = os.path.join(base_path, "{}/{}/USDA_{}_County_{}.csv")
    
    all_counties = set()
    counties_with_full_data = set()
    
    for year in range(start_year, end_year + 1):
        file_path = path_template.format(folder_name, year, file_prefix, year)
        
        try:
            data = pd.read_csv(file_path)
            # Filter for the specified state
            state_data = data[data['state_ansi'] == int(state_fips)]
            year_counties = set(state_data['state_ansi'].astype(str).str.zfill(2) + 
                                state_data['county_ansi'].astype(str).str.zfill(3))
            
            if year == start_year:
                all_counties = year_counties
            else:
                counties_with_full_data = all_counties.intersection(year_counties)
                all_counties = counties_with_full_data
        
        except FileNotFoundError:
            print(f"File not found: {file_path}")
            return set()  # If any file is missing, return an empty set
    
    return counties_with_full_data

# Example usage:
crop_type = "Corn"
state_fips = "17"  
counties_with_full_data = get_counties_with_full_data(crop_type, state_fips, 2017, 2022)
print(f"Number of counties in {state_fips} with full {crop_type} data from 2017 to 2022: {len(counties_with_full_data)}")
print("FIPS codes:", sorted(counties_with_full_data),'\n')
crop_type = "Cotton"
state_fips = "01"  
counties_with_full_data = get_counties_with_full_data(crop_type, state_fips, 2017, 2022)
print(f"Number of counties in {state_fips} with full {crop_type} data from 2017 to 2022: {len(counties_with_full_data)}")
print("FIPS codes:", sorted(counties_with_full_data),'\n')
crop_type = "WinterWheat"
state_fips = "17"  
counties_with_full_data = get_counties_with_full_data(crop_type, state_fips, 2017, 2022)
print(f"Number of counties in {state_fips} with full {crop_type} data from 2017 to 2022: {len(counties_with_full_data)}")
print("FIPS codes:", sorted(counties_with_full_data),'\n')
crop_type = "Soybeans"
state_fips = "17"  
counties_with_full_data = get_counties_with_full_data(crop_type, state_fips, 2017, 2022)
print(f"Number of counties in {state_fips} with full {crop_type} data from 2017 to 2022: {len(counties_with_full_data)}")
print("FIPS codes:", sorted(counties_with_full_data))

Number of counties in 17 with full Corn data from 2017 to 2022: 54
FIPS codes: ['17007', '17011', '17015', '17017', '17019', '17021', '17025', '17027', '17033', '17037', '17049', '17053', '17055', '17057', '17059', '17061', '17063', '17073', '17075', '17077', '17081', '17085', '17089', '17093', '17095', '17101', '17103', '17105', '17107', '17113', '17115', '17117', '17119', '17121', '17123', '17133', '17135', '17139', '17141', '17143', '17147', '17157', '17163', '17167', '17169', '17173', '17175', '17177', '17179', '17189', '17193', '17195', '17201', '17203'] 

Number of counties in 01 with full Cotton data from 2017 to 2022: 18
FIPS codes: ['01003', '01015', '01019', '01031', '01039', '01045', '01047', '01053', '01061', '01067', '01069', '01077', '01079', '01083', '01089', '01097', '01099', '01117'] 

Number of counties in 17 with full WinterWheat data from 2017 to 2022: 26
FIPS codes: ['17011', '17013', '17023', '17025', '17027', '17033', '17037', '17047', '17049', '17067', '17083', 